<a href="https://colab.research.google.com/github/szambetti/INVEST_IO/blob/master/stock_scraping_investing_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Questo notebook scarica i vari calendari da investing, le date sono settabili come riportato sotto



CALENDARIO ECONOMICO

In [1]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
import pandas
import json

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

     |████████████████████████████████| 911kB 4.5MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: use options instead of chrome_options


In [2]:
#importa la data di oggi nel formato corretto
from datetime import date
import time
today = date.today()
today_us_format = today.strftime("%m/%d/%Y")

In [3]:
#function per cliccare pulsanti su chrome .click() di selenium non funziona
def click_on_button_id(x):
  element = wd.find_element_by_id(x)
  wd.execute_script("arguments[0].click();", element)

In [4]:
#CALENDARIO ECONOMICO

url = 'https://www.investing.com/economic-calendar'

wd.set_window_size(1920,1080)
wd.maximize_window()
wd.get(url)

#apply filter on dates, scarica sempre da startdate a oggi, !!! FORMATO DATE US !!!
startdate = '08/01/2020'

#clicca sui pulsanti per filtrare la data
click_on_button_id('datePickerToggleBtn')
click_on_button_id('startDate')
wd.find_element_by_id("startDate").clear()
wd.find_element_by_id("startDate").send_keys(startdate)
click_on_button_id('endDate')
wd.find_element_by_id("endDate").clear()
wd.find_element_by_id("endDate").send_keys(today_us_format)
click_on_button_id("applyBtn")

#wait for the apply button to be effective
time.sleep(5)


#scrolla fino alla fine della pagina, aspetta questo timeout per caricare i dati
SCROLL_PAUSE_TIME = 4

#scrollare main method
last_height = wd.execute_script("return document.body.scrollHeight")
while True:
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = wd.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height


list_of_news = []

#launch main scraping loop
for tr in wd.find_elements_by_xpath('//*[@id="economicCalendarData"]/tbody//tr'):
    tds = tr.find_elements_by_tag_name('td')
    list_of_news_data=[]
    for td in tds:
      if td.text != '':
        list_of_news_data.append(td.text)
      elif td.get_attribute('title') != '':
        list_of_news_data.append(td.get_attribute('title'))
      else:
        list_of_news_data.append('')
    if len(list_of_news_data) == 1:
        date = list_of_news_data[0]
    elif len(list_of_news_data) == 4:
      list_of_news.append({'date' : date, 'time': list_of_news_data[0], 
                            'currency' : list_of_news_data[1],
                            'importance' : list_of_news_data[2], 
                            'event' : list_of_news_data[3],
                            'actual' : '',
                            'expected' : '',
                            'previous' : ''})
    else:
      list_of_news.append({'date' : date, 'time': list_of_news_data[0], 
                            'currency' : list_of_news_data[1],
                            'importance' : list_of_news_data[2], 
                            'event' : list_of_news_data[3],
                            'actual' : list_of_news_data[4],
                            'expected' : list_of_news_data[5],
                            'previous' : list_of_news_data[6]})

In [5]:
from pandas import DataFrame
df = DataFrame (list_of_news)
df.head()
df.to_csv('sr_economic_calendar.csv')

In [6]:
df.head(15)

,date,time,currency,importance,event,actual,expected,previous
0,"Saturday, August 1, 2020",All Day,,Holiday,India - Eid al-Adha,,,
1,"Sunday, August 2, 2020",18:30,AUD,Moderate Volatility Expected,AIG Manufacturing Index (Jul),53.5,,51.5
2,"Sunday, August 2, 2020",19:50,JPY,Moderate Volatility Expected,GDP (YoY),-2.2%,-4.1%,-2.2%
3,"Sunday, August 2, 2020",19:50,JPY,High Volatility Expected,GDP (QoQ),-0.6%,-1.1%,-0.6%
4,"Sunday, August 2, 2020",19:50,JPY,Low Volatility Expected,GDP Capital Expenditure (QoQ),1.7%,,1.9%
5,"Sunday, August 2, 2020",19:50,JPY,Low Volatility Expected,GDP External Demand (QoQ),-0.2%,,-0.2%
6,"Sunday, August 2, 2020",19:50,JPY,Low Volatility Expected,GDP Price Index (YoY),0.9%,,0.9%
7,"Sunday, August 2, 2020",19:50,JPY,Low Volatility Expected,GDP Private Consumption (QoQ),-0.8%,,-0.8%
8,"Sunday, August 2, 2020",20:30,KRW,Low Volatility Expected,Nikkei Manufacturing PMI (Jul),46.9,,43.4
9,"Sunday, August 2, 2020",20:30,JPY,Low Volatility Expected,Manufacturing PMI (Jul),45.2,42.6,40.1


CALENDARIO EARNINGS

In [9]:
#CALENDARIO EARNINGS

url = 'https://www.investing.com/earnings-calendar/'

wd.set_window_size(1920,1080)
wd.maximize_window()
wd.get(url)

#apply filter on dates, scarica sempre da startdate a oggi, !!! FORMATO DATE US !!!
startdate = '08/01/2020'

#clicca sui pulsanti per filtrare la data
click_on_button_id('datePickerToggleBtn')
click_on_button_id('startDate')
wd.find_element_by_id("startDate").clear()
wd.find_element_by_id("startDate").send_keys(startdate)
click_on_button_id('endDate')
wd.find_element_by_id("endDate").clear()
wd.find_element_by_id("endDate").send_keys(today_us_format)
click_on_button_id("applyBtn")

#wait for the apply button to be effective
time.sleep(5)

wd.save_screenshot('screen.png')

output_list = []

#main scraper loop

for tr in wd.find_elements_by_xpath('//*[@id="earningsCalendarData"]/tbody//tr'):
    tds = tr.find_elements_by_tag_name('td')
    temp_list = []
    for td in tds:
      if td.text != '':
        temp_list.append(td.text)
      elif td.get_attribute('title') != '':
        temp_list.append(td.get_attribute('title'))
      else:
        temp_list.append('')
      if len(temp_list
      output_list.append({'date' : date, 'company': temp_list[1], 
                            'EPS' : temp_list[2],
                            '/ Forecast' : temp_list[2], 
                            'Revenue' : temp_list[3],
                            '/ Forecast' : temp_list[4],
                            'Market Cap' : temp_list[5]})

print(output_list)

IndexError: ignored